In [8]:
# Env
import os
from pathlib import Path
import pandas as pd
import numpy as np
import re


DATASET_DIR = Path('/mnt/f/spatial_data/probe')
RUNID = 'Example_dataset'
workdir = DATASET_DIR / RUNID
resultsdir = os.path.join(workdir, 'results')
organism = 'human'

In [ ]:
gene_info = pd.read_excel(os.path.join(workdir, "marker_gene_list.xlsx"))
gene_info['gene'] = gene_info['gene name']
if organism == 'mouse': gene_info['gene'] = gene_info['gene'].str.capitalize()
elif organism == 'human': gene_info['gene'] = gene_info['gene'].str.upper()
gene_list = [_.strip() for _ in gene_info['gene'].unique() if _!=0]
print(len(gene_list))
gene_info.head()

In [ ]:
def adjust_gene_name(gene_name, gene_list):
    gene_list = [x.upper() for x in gene_list]
    match = re.search(r'(.+)-(\d+)$', gene_name)
    if match:
        base_gene_name = match.group(1)
        if base_gene_name.upper() in gene_list or gene_name.upper() in gene_list: return base_gene_name
        else: return gene_name
    else: return gene_name


result = pd.DataFrame()
for dir in os.listdir(os.path.join(resultsdir)):
    try: result = pd.concat([result, pd.read_excel(os.path.join(resultsdir, dir, "probes_wanted.xlsx"), index_col=0)])
    except: continue

result["gene_name"] = [adjust_gene_name(_, gene_list) for _ in result["gene_name"]]
result = result.sort_values(["gene_name", "pos"])
result.drop_duplicates(subset=["bds"], keep="first", inplace=True)
result.head()

In [11]:
result.to_excel(os.path.join(workdir, 'gene_binding_site.xlsx'))

result["gene_name"] = result["gene_name"].str.upper()
to_search = [gene for gene in gene_list if len(result[result["gene_name"] == gene]) < 1]
with open(os.path.join(workdir, "to_search.txt"), "w") as f:
    for line in to_search: f.write(line + "\n")